<a href="https://colab.research.google.com/github/roulaaa/paraA3/blob/main/cudaHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!echo "c.NotebookApp.iopub_data_rate_limit = 100000000" >> ~/.jupyter/jupyter_notebook_config.py
!pkill -f "jupyter-notebook"


In [45]:
#===sequential=======

%%writefile seq.c 


#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define ROWS_A 70
#define COLS_A 80
#define ROWS_B COLS_A
#define COLS_B 70

// Function to multiply the matrices
void matrixMul(const int *a, const int *b, int *c) {
  for(int row = 0; row < ROWS_A; row++) {
    for(int col = 0; col < COLS_B; col++) {
      int tmp = 0;
      for (int i = 0; i < COLS_A; i++) {
        tmp += a[row * COLS_A + i] * b[i * COLS_B + col];
      }
      c[row * COLS_B + col] = tmp;
    }
  }
}

//print the matrix
void printMatrix(const int *matrix, int rows, int cols) {
  for (int row = 0; row < rows; row++) {
    for (int col = 0; col < cols; col++) {
      printf("%d ", matrix[row * cols + col]);
    }
    printf("\n");
  }
}

int verify_result(int *a, int *b, int *c) {
  for (int i = 0; i < ROWS_A; i++) {
    for (int j = 0; j < COLS_B; j++) {
      int tmp = 0;
      for (int k = 0; k < COLS_A; k++) {
        tmp += a[i * COLS_A + k] * b[k * COLS_B + j];
      }

      // If the operation did not succeed, return false
      if (tmp != c[i * COLS_B + j]) {
        return 0;
      }
    }
  }

//return true 
  return 1;
}

int main() {
  size_t bytes_a = ROWS_A * COLS_A * sizeof(int);
  size_t bytes_b = ROWS_B * COLS_B * sizeof(int);
  size_t bytes_c = ROWS_A * COLS_B * sizeof(int);

  int *h_a = (int*)malloc(bytes_a);
  int *h_b = (int*)malloc(bytes_b);
  int *h_c = (int*)malloc(bytes_c);

  for (int i = 0; i < ROWS_A * COLS_A; i++) {
    h_a[i] = rand() % 100;
  }
  
  for (int i = 0; i < ROWS_B * COLS_B; i++) {
    h_b[i] = rand() % 100;
  }

  clock_t start, end;
  double cpu_time_used;

  start = clock();

  matrixMul(h_a, h_b, h_c);

  end = clock();
  cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;

  printf("Matrix A:\n");
  printMatrix(h_a, ROWS_A, COLS_A);
  printf("\nMatrix B:\n");
  printMatrix(h_b, ROWS_B, COLS_B);
  printf("\nMatrix C (result):\n");
  printMatrix(h_c, ROWS_A, COLS_B);
  
  int success = verify_result(h_a, h_b, h_c);

  if (success) {
    printf("COMPLETED SUCCESSFULLY\n");
  } else {
    printf("ERROR: THE OPERATION DID NOT SUCCEED\n");
  }
  
  printf("Time taken: %f seconds\n", cpu_time_used);

  free(h_a);
  free(h_b);
  free(h_c);

  return 0;
}



Overwriting seq.c


In [46]:
%%shell 
gcc seq.c -o seq -lm
./seq

Matrix A:
83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 36 5 46 29 13 57 24 95 82 45 14 67 34 64 43 50 
87 8 76 78 88 84 3 51 54 99 32 60 76 68 39 12 26 86 94 39 95 70 34 78 67 1 97 2 17 92 52 56 1 80 86 41 65 89 44 19 40 29 31 17 97 71 81 75 9 27 67 56 97 53 86 65 6 83 19 24 28 71 32 29 3 19 70 68 8 15 40 49 96 23 18 45 46 51 21 55 
79 88 64 28 41 50 93 0 34 64 24 14 87 56 43 91 27 65 59 36 32 51 37 28 75 7 74 21 58 95 29 37 35 93 18 28 43 11 28 29 76 4 43 63 13 38 6 40 4 18 28 88 69 17 17 96 24 43 70 83 90 99 72 25 44 90 5 39 54 86 69 82 42 64 97 7 55 4 48 11 
22 28 99 43 46 68 40 22 11 10 5 1 61 30 78 5 20 36 44 26 22 65 8 16 82 58 24 37 62 24 0 36 52 99 79 50 68 71 73 31 81 30 33 94 60 63 99 81 99 96 59 73 13 68 90 95 26 66 84 40 90 84 76 42 36 7 45 56 79 18 87 12 48 72 59 9 36 10 42 87 
6 1 13 72 21 55 19 99 21 4 39 11 40 67 5 28 27 50 84 

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [6]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cen3nghg
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cen3nghg
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [7]:
%load_ext nvcc_plugin


The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [49]:
#===CUDA===
%%cu

#include <time.h>
#include <stdio.h>
#include <stdlib.h>

#define ROWS_A 70
#define COLS_A 80
#define ROWS_B COLS_A
#define COLS_B 70

#define SHMEM_SIZE (1 << 10)

//CUDA Kernel function to multiply the matrices
__global__ void matrixMul(const int *a, const int *b, int *c) {
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  __shared__ int s_a[SHMEM_SIZE];
  __shared__ int s_b[SHMEM_SIZE];

  int tmp = 0;

  for (int i = 0; i < COLS_A; i += blockDim.x) {
    s_a[threadIdx.y * blockDim.x + threadIdx.x] = a[row * COLS_A + i + threadIdx.x];
    s_b[threadIdx.y * blockDim.x + threadIdx.x] = b[i * COLS_B + threadIdx.y * COLS_B + col];

    __syncthreads();

    for (int j = 0; j < blockDim.x; j++) {
      tmp += s_a[threadIdx.y * blockDim.x + j] * s_b[j * blockDim.x + threadIdx.x];
    }

    __syncthreads();
  }

  c[row * COLS_B + col] = tmp;
}

int verify_result(int *a, int *b, int *c) {
  for (int i = 0; i < ROWS_A; i++) {
    for (int j = 0; j < COLS_B; j++) {
      int tmp = 0;
      for (int k = 0; k < COLS_A; k++) {
        tmp += a[i * COLS_A + k] * b[k * COLS_B + j];
      }

      // If the operation did not succeed, return false
      if (tmp != c[i * COLS_B + j]) {
        return 0;
      }
    }
  }

  // If the operation succeeded, return true
  return 1;
}

void print_matrix(int *mat, int rows, int cols) {
  for (int i = 0; i < rows; ++i) {
    for (int j = 0; j < cols; ++j) {
      printf("%d ", mat[i * cols + j]);
    }
    printf("\n");
  }
}

int main() {
  size_t bytes_a = ROWS_A * COLS_A * sizeof(int);
  size_t bytes_b = ROWS_B * COLS_B * sizeof(int);
  size_t bytes_c = ROWS_A * COLS_B * sizeof(int);

  int *h_a = (int*)malloc(bytes_a);
  int *h_b = (int*)malloc(bytes_b);
  int *h_c = (int*)malloc(bytes_c);

  for (int i = 0; i < ROWS_A * COLS_A; i++) {
    h_a[i] = rand() % 100;
  }

  for (int i = 0; i < ROWS_B * COLS_B; i++) {
    h_b[i] = rand() % 100;
  }

  int *d_a, *d_b, *d_c;
  cudaMalloc(&d_a, bytes_a);
  cudaMalloc(&d_b, bytes_b);
  cudaMalloc(&d_c, bytes_c);

  cudaMemcpy(d_a, h_a, bytes_a, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, bytes_b, cudaMemcpyHostToDevice);

  int THREADS = 10;
  int BLOCKS_A = ROWS_A / THREADS;
  int BLOCKS_B = COLS_B / THREADS;

  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS_B, BLOCKS_A);

  int totalThreads = THREADS * THREADS * BLOCKS_B * BLOCKS_A;
  printf("Total threads: %d\n", totalThreads);
  clock_t start, end;
  double cpu_time_used;
  
  start = clock();

  matrixMul<<<blocks, threads>>>(d_a, d_b, d_c);

  cudaMemcpy(h_c, d_c, bytes_c, cudaMemcpyDeviceToHost);

  end = clock();
  cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;

  printf("Matrix A:\n");
  print_matrix(h_a, ROWS_A, COLS_A);
  printf("\nMatrix B:\n");
  print_matrix(h_b, ROWS_B, COLS_B);
  printf("\nMatrix C (result):\n");
  print_matrix(h_c, ROWS_A, COLS_B);

  int success = verify_result(h_a, h_b, h_c);

  if (success) {
    printf("COMPLETED SUCCESSFULLY\n");
  } else {
    printf("ERROR WITH COMPUTATION\n");
  }

  printf("Time taken: %f seconds\n", cpu_time_used);

  free(h_a);
  free(h_b);
  free(h_c);
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  return 0;
}


Total threads: 4900
Matrix A:
83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 36 5 46 29 13 57 24 95 82 45 14 67 34 64 43 50 
87 8 76 78 88 84 3 51 54 99 32 60 76 68 39 12 26 86 94 39 95 70 34 78 67 1 97 2 17 92 52 56 1 80 86 41 65 89 44 19 40 29 31 17 97 71 81 75 9 27 67 56 97 53 86 65 6 83 19 24 28 71 32 29 3 19 70 68 8 15 40 49 96 23 18 45 46 51 21 55 
79 88 64 28 41 50 93 0 34 64 24 14 87 56 43 91 27 65 59 36 32 51 37 28 75 7 74 21 58 95 29 37 35 93 18 28 43 11 28 29 76 4 43 63 13 38 6 40 4 18 28 88 69 17 17 96 24 43 70 83 90 99 72 25 44 90 5 39 54 86 69 82 42 64 97 7 55 4 48 11 
22 28 99 43 46 68 40 22 11 10 5 1 61 30 78 5 20 36 44 26 22 65 8 16 82 58 24 37 62 24 0 36 52 99 79 50 68 71 73 31 81 30 33 94 60 63 99 81 99 96 59 73 13 68 90 95 26 66 84 40 90 84 76 42 36 7 45 56 79 18 87 12 48 72 59 9 36 10 42 87 
6 1 13 72 21 55 19 99 21 4 39 11 